In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import websocket
import json
import os
import configparser
from binance.client import Client
from binance.enums import *
# from enum import Enum
# import math
# import time

In [ ]:
config = configparser.ConfigParser()
config.read("./env.cfg")


In [ ]:
client = Client(config['binance-testnet']['api_key'], config['binance-testnet']['api_secret'], testnet=True)

In [ ]:
client.get_asset_balance(asset='BTC')

# {'asset': 'BTC', 'free': '1.00000000', 'locked': '0.00000000'}

In [ ]:
client.get_asset_balance(asset='USDT')

# {'asset': 'USDT', 'free': '9999.01016524', 'locked': '0.00000000'}

In [ ]:
client.get_my_trades(symbol='BTCUSDT')

# [
#     {
#         'symbol': 'BTCUSDT',
#         'id': 501256,
#         'orderId': 1793805,
#         'orderListId': -1,
#         'price': '24189.74000000',
#         'qty': '0.08202800',
#         'quoteQty': '1984.23599272',
#         'commission': '0.00000000',
#         'commissionAsset': 'BTC',
#         'time': 1659971422618,
#         'isBuyer': True,
#         'isMaker': False,
#         'isBestMatch': True
#     }
# ]

In [ ]:
def order(side, qty: float) -> bool:
    ''' Place a buy or sell order
    
    arguments: 
    - side: should be 'BUY' or 'SELL'

    returns bool, True in case of success
    '''
    try:
        print(f"place {side} order")
        order = client.create_order(
            symbol='BTCUSDT',
            side=side,
            type=ORDER_TYPE_MARKET,
            quantity=qty,
        )
        print(order)

    except Exception as e:
        print(f"Error: {e}")
        return False

    return True 

def buy(qty: float = 0.01):
    order(SIDE_BUY, qty)

def sell(qty: float = 0.01):
    order(SIDE_SELL, qty)

In [ ]:
# buy()
sell()


In [ ]:
WS_URL = 'wss://stream.binance.us:9443/ws/btcusdt@kline_1m'

class OnlineTrading(object):

    def __init__(self, momentum: int = 1):
        self.data = pd.DataFrame()
        self.momentum = momentum
        self.position = 0
        self.min_length = momentum + 2

    def run(self):
        ws = websocket.WebSocketApp(
            WS_URL,
            on_open=self.__on_open,
            on_message=self.__on_message,
            on_close=self.__on_close
        )
        ws.run_forever()

    def __on_open(self, ws):
        print("Connection established")

    def __on_close(self, ws, code, msg):
        print(f"Connection closed, close position by selling.")
        self.sell()

    def __on_message(self, ws, message):
        data = json.loads(message)
        close = float(data['k']['c'])
        date = dt.datetime.fromtimestamp(int(int(data['E']) / 1000))
        is_candle_closed = True #bool(data['k']['x'])

        if is_candle_closed:
            # create & append new candle to self.data
            new_candle = pd.DataFrame({ "price": close }, index=[date])
            new_candle.index.name = 'Date'
            self.data = pd.concat([self.data, new_candle])

            dr = self.data.resample('15s', label='right').last()
            dr['return'] = np.log(dr['price'] / dr['price'].shift(1))
            dr['momentum'] = np.sign(dr['return'].rolling(self.momentum).mean())
        
            if len(dr) > self.min_length:
                if dr['momentum'].iloc[-2] > 0 and self.position == 0:
                    self.buy()
                elif dr['momentum'].iloc[-2] < 0 and self.position == 1:
                    self.sell()


    def buy(self):
        print("buy! buy! buy!")
        self.position = 1

    def sell(self):
        print("sell! sell! sell!")
        self.position = 0

bot = OnlineTrading()
bot.run()